In [ ]:
import pandas as pd
kbar_folder = '/home/syl/Code/FinRL_Market_Simulator/data/kbar_m5'

In [37]:
import os

df_old = pd.read_csv(r"C:\Users\songyunlu\Downloads\minute_bars_2017_2020_202405101655.csv")
df_new = pd.read_csv(r"C:\Users\songyunlu\Downloads\minute_bars_latest_202405101655.csv")



# 合并从数据库导出的 m5 kbar数据
df_kbar_5 = pd.concat([df_old, df_new])
df_kbar_5_group = df_kbar_5.groupby('SecuCode')
for stock_code, df_kbar in df_kbar_5_group:
    df_kbar.drop_duplicates().to_csv(os.path.join(kbar_folder, f'{stock_code}.csv'), index=False)

In [9]:
import os

# 补充 成交量分布数据以及过去60天的成交量平均分布数据
backdays = 60

for file in os.listdir(kbar_folder):
    df = pd.read_csv(os.path.join(kbar_folder, file))
    tmp_list = []
    for trading_day, sub_df in df.groupby('TradingDay'):
        total_vol = sub_df['Volume'].sum()
        sub_df[f'vol_percentage'] = sub_df['Volume'] / total_vol
        tmp_list.append(sub_df)
    df = pd.concat(tmp_list)
    
    tmp_list = []
    for update_time, sub_df in df.groupby('UpdateTime'):
        sub_df = sub_df.sort_values('TradingDay')
        sub_df[f'past_{backdays}_day_avg_vol'] = sub_df['vol_percentage'].rolling(window=backdays).mean().shift(1)
        tmp_list.append(sub_df)
    df = pd.concat(tmp_list).sort_values(by=['TradingDay','UpdateTime'])
    df.to_csv(os.path.join(kbar_folder, file), index=False)
    
    